In [1]:
pip install python-binance 

Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: regex
    Found existing installation: regex 2022.3.15
    Uninstalling regex-2022.3.15:
      Successfully uninstalled regex-2022.3.15


In [2]:
pip install ta 

  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28986 sha256=7de2d4d6843e6aef3c595732ab8e25b2b8698646839000a472805d96f116b774
  Stored in directory: c:\users\vaio\appdata\local\pip\cache\wheels\fa\f5\2a\22332fca050eb5e48116f8534cd267d50cffcd1e7581606150
Successfully built ta
Note: you may need to restart the kernel to use updated packages.


# Import define Client and Download Data

In [21]:
import pandas as pd
from binance.client import Client  
import ta
klinesT = Client().get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1HOUR, "01 January 2016")#recuperer les donnees heure par heure depuis 2016
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])#construction du dataframe

In [22]:
print(df)

           timestamp            open            high             low  \
0      1502942400000   4261.48000000   4313.62000000   4261.32000000   
1      1502946000000   4308.83000000   4328.69000000   4291.37000000   
2      1502949600000   4330.29000000   4345.45000000   4309.37000000   
3      1502953200000   4316.62000000   4349.99000000   4287.41000000   
4      1502956800000   4333.32000000   4377.85000000   4333.32000000   
...              ...             ...             ...             ...   
43058  1658404800000  22607.92000000  22850.00000000  22542.14000000   
43059  1658408400000  22802.82000000  22812.77000000  22561.27000000   
43060  1658412000000  22702.30000000  22761.77000000  22341.46000000   
43061  1658415600000  22555.31000000  22679.24000000  22520.17000000   
43062  1658419200000  22639.37000000  22980.00000000  22632.72000000   

                close          volume     close_time            quote_av  \
0       4308.83000000     47.18100900  1502945999999     20

# Clean DATA

In [23]:
#suppression des colonnes dont on a pas besoin
del df['ignore']
del df['close_time']
del df['quote_av']
del df['trades']
del df['tb_base_av']
del df['tb_quote_av']

df['close'] = pd.to_numeric(df['close'])
df['high'] = pd.to_numeric(df['high'])
df['low'] = pd.to_numeric(df['low'])
df['open'] = pd.to_numeric(df['open'])
#transformer les timepstamp en timestamp lisible
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']
print(df)

                         open      high       low     close          volume
timestamp                                                                  
2017-08-17 04:00:00   4261.48   4313.62   4261.32   4308.83     47.18100900
2017-08-17 05:00:00   4308.83   4328.69   4291.37   4315.32     23.23491600
2017-08-17 06:00:00   4330.29   4345.45   4309.37   4324.35      7.22969100
2017-08-17 07:00:00   4316.62   4349.99   4287.41   4349.99      4.44324900
2017-08-17 08:00:00   4333.32   4377.85   4333.32   4360.69      0.97280700
...                       ...       ...       ...       ...             ...
2022-07-21 12:00:00  22607.92  22850.00  22542.14  22800.20   9839.15857000
2022-07-21 13:00:00  22802.82  22812.77  22561.27  22704.96  10543.33923000
2022-07-21 14:00:00  22702.30  22761.77  22341.46  22558.20  12186.72997000
2022-07-21 15:00:00  22555.31  22679.24  22520.17  22642.34   6414.29306000
2022-07-21 16:00:00  22639.37  22980.00  22632.72  22975.31   4663.00642000

[43063 rows

# Define Indicators SMA200 and SMA600

In [24]:
df['SMA200'] = ta.trend.sma_indicator(df['close'], 200)
df['SMA600'] = ta.trend.sma_indicator(df['close'], 600)
print(df)

                         open      high       low     close          volume  \
timestamp                                                                     
2017-08-17 04:00:00   4261.48   4313.62   4261.32   4308.83     47.18100900   
2017-08-17 05:00:00   4308.83   4328.69   4291.37   4315.32     23.23491600   
2017-08-17 06:00:00   4330.29   4345.45   4309.37   4324.35      7.22969100   
2017-08-17 07:00:00   4316.62   4349.99   4287.41   4349.99      4.44324900   
2017-08-17 08:00:00   4333.32   4377.85   4333.32   4360.69      0.97280700   
...                       ...       ...       ...       ...             ...   
2022-07-21 12:00:00  22607.92  22850.00  22542.14  22800.20   9839.15857000   
2022-07-21 13:00:00  22802.82  22812.77  22561.27  22704.96  10543.33923000   
2022-07-21 14:00:00  22702.30  22761.77  22341.46  22558.20  12186.72997000   
2022-07-21 15:00:00  22555.31  22679.24  22520.17  22642.34   6414.29306000   
2022-07-21 16:00:00  22639.37  22980.00  22632.72  2

# Make Backtest

In [25]:
usdt = 1000
btc = 0
lastIndex = df.first_valid_index()

for index, row in df.iterrows():
  if df['SMA200'][lastIndex] > df['SMA600'][lastIndex] and usdt > 10:
    btc = usdt / df['close'][index] 
    btc = btc - 0.0007 * btc
    usdt = 0
    print("Buy BTC at",df['close'][index],'$', index)

  if df['SMA200'][lastIndex] < df['SMA600'][lastIndex] and btc > 0.0001:
    usdt = btc * df['close'][index]
    usdt = usdt - 0.0007 * usdt
    btc = 0
    print("Sell BTC at",df['close'][index],'$', index)
  lastIndex = index

Buy BTC at 4191.53 $ 2017-09-11 11:00:00
Sell BTC at 4259.44 $ 2017-09-12 13:00:00
Buy BTC at 4302.02 $ 2017-10-01 08:00:00
Sell BTC at 16351.02 $ 2017-12-27 05:00:00
Buy BTC at 14219.61 $ 2018-01-10 06:00:00
Sell BTC at 14195.0 $ 2018-01-15 12:00:00
Buy BTC at 10940.0 $ 2018-02-19 17:00:00
Sell BTC at 8704.0 $ 2018-03-11 07:00:00
Buy BTC at 8018.98 $ 2018-04-16 21:00:00
Sell BTC at 8665.97 $ 2018-05-14 00:00:00
Buy BTC at 6584.36 $ 2018-07-07 10:00:00
Sell BTC at 7087.99 $ 2018-08-07 11:00:00
Buy BTC at 6902.74 $ 2018-08-28 04:00:00
Sell BTC at 6355.0 $ 2018-09-11 03:00:00
Buy BTC at 6669.36 $ 2018-09-27 19:00:00
Sell BTC at 6309.02 $ 2018-10-12 22:00:00
Buy BTC at 6573.58 $ 2018-10-20 10:00:00
Sell BTC at 6491.85 $ 2018-10-28 10:00:00
Buy BTC at 4125.4 $ 2018-12-24 04:00:00
Sell BTC at 3588.24 $ 2019-01-13 02:00:00
Buy BTC at 3626.58 $ 2019-02-12 04:00:00
Sell BTC at 7660.98 $ 2019-06-09 18:00:00
Buy BTC at 9105.21 $ 2019-06-17 06:00:00
Sell BTC at 9431.52 $ 2019-07-17 04:00:00
Buy B

# Comparaison Final result with buy and hold result

In [26]:
fr = usdt + btc * df['close'].iloc[-1]
print("Final result",fr,'USDT')

Final result 24943.980041956467 USDT


In [27]:
print("Buy and hold result", (1000 / df['close'].iloc[0]) * df['close'].iloc[-1],'USDT')

Buy and hold result 5332.145849337292 USDT


we notice that with our bot we made more benefit than if we bought in 2016 and sold today

-with bot --------->23943$ benefits


-buy and hold strategy --------->4332$ benefits